In [3]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

# Function to sample queries
def sample_queries(df, frac):
    srch_ids = pd.Series(df["srch_id"].unique()).sample(frac=frac, random_state=42)
    return df.loc[df.srch_id.isin(srch_ids)]

in_train = pd.read_csv("data/training_set_VU_DM_2014.csv")
#in_test = pd.read_csv("data/testing_set_VU_DM_2014.csv")
#in_test = pd.read_csv("data/kaggle_test.csv")
in_test = in_train.drop(["click_bool", "booking_bool", "position", "gross_bookings_usd"], axis=1).head()

# Speed up execution
#in_train = sample_queries(in_train, 0.1)

in_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [4]:
# Add mean, median and std per property for a variety of numericals
def add_mean_median_std_per_factor(df, factor, numerical, prefix):
    grouped = df.groupby(factor)[numerical]
    mean_by_factor = grouped.mean().to_dict()
    median_by_factor = grouped.median().to_dict()
    std_by_factor = grouped.std().to_dict()
    
    df[prefix + "_mean_" + numerical] = df[factor].map(lambda f: mean_by_factor[f])
    df[prefix + "_median_" + numerical] = df[factor].map(lambda f: median_by_factor[f])
    df[prefix + "_std_" + numerical] = df[factor].map(lambda f: std_by_factor[f])
    
for df in [in_train, in_test]:
    f = add_mean_median_std_per_factor
    f(df, "srch_id", "price_usd", "srch")
    f(df, "srch_id", "prop_starrating", "srch")
    f(df, "srch_id", "prop_review_score", "srch")
    
    # These features don't seem to work well, keeping it in for now though
    f(df, "srch_destination_id", "price_usd", "dest")
    f(df, "srch_destination_id", "prop_starrating", "dest")
    f(df, "srch_destination_id", "prop_review_score", "dest")
    
    f(df, "prop_id", "price_usd", "prop")
    f(df, "prop_id", "prop_review_score", "prop")
    
in_train.head()

KeyboardInterrupt: 

In [ ]:
# Normalize price per query
for df in [in_train, in_test]:
    df.price_usd = df.groupby("srch_id").price_usd.transform(lambda x: (x - x.mean()) / x.std())


In [ ]:
# Split on search ids
srch_ids = in_train["srch_id"].unique()
srch_ids_train, srch_ids_test_val = train_test_split(srch_ids, test_size=0.2, random_state = 42)
srch_ids_test, srch_ids_val = train_test_split(srch_ids_test_val, test_size=0.5, random_state = 42)

_train = in_train.loc[in_train.srch_id.isin(srch_ids_train)]
_val = in_train.loc[in_train.srch_id.isin(srch_ids_val)]
_test = in_train.loc[in_train.srch_id.isin(srch_ids_test)]

In [ ]:
# Estimate probability of being clicked and booked
#grouped = _train.groupby("prop_id")    
#click_prob_by_prop = grouped.click_bool.mean().to_dict()
#book_prob_by_prop = grouped.booking_bool.mean().to_dict()

#mean_click_prob = in_train.click_bool.mean()
#mean_book_prob = in_train.booking_bool.mean()
    
#for df in [_train, _val, _test, in_test]:
#    df["click_probability"] = df.prop_id.map(lambda p: click_prob_by_prop.setdefault(p, mean_click_prob))
#    df["booking_probability"] = df.prop_id.map(lambda p: book_prob_by_prop.setdefault(p, mean_book_prob))

In [ ]:
# Undersample a series so that there are as many non-clicked items as clicked and booked
def downsample_series(s):
    s1 = s.loc[s.click_bool == 1]
    s2 = s.loc[s.click_bool == 0]
    
    if (len(s2) == 0):
        return s1
    
    s2 = s2.sample(frac=min(1, len(s1)/len(s)))
    return s1.append(s2).sort_index()

_train_undersampled = _train.groupby("srch_id").apply(downsample_series).reset_index(drop=True)

In [ ]:
# Save to CSV
#_train.to_csv("data/processed/_train.csv", index=False)
_val.to_csv("data/processed/_val.csv", index=False)
_test.to_csv("data/processed/_test.csv", index=False)
_train_undersampled.to_csv("data/processed/_train_undersampled.csv", index=False)
in_test.to_csv("data/processed/kaggle_test.csv", index=False)

In [ ]:
in_test.head()

In [ ]:
in_train.columns